<a href="https://colab.research.google.com/github/ThatoMkoni/ASU_CNN_TENSORFLOW/blob/main/ASU_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow
!pip install -q keras

In [ ]:
!wget https://github.com/saraalemadi/DroneAudioDataset/archive/refs/heads/master.zip

!unzip /content/master.zip

In [5]:
import tensorflow as tf
import keras
from keras import layers, models


import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [6]:
def load_wav_mono(filename : str):
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)  # type: ignore
    wav = tf.squeeze(wav, axis=1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    return wav

def preprocess(file_path : str, label : str):
    wav = load_wav_mono(file_path)
    wav = wav[:48000]
    zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)  # type: ignore
    wav = tf.concat([zero_padding, wav], 0)
    spectogram = tf.signal.stft(wav, frame_length=160, frame_step = 16)
    spectogram = tf.abs(spectogram)
    print(spectogram.shape)
    spectogram = tf.expand_dims(spectogram, axis=2)
    print(spectogram.shape)
    return spectogram, label


In [7]:
# ensures reproducibliity as neurons are set at a known seed
seed = 7
tf.random.set_seed(seed)
np.random.seed(seed)


In [ ]:

DATASET_PATH = '/content/DroneAudioDataset-master/Binary_Drone_Audio'
yes_drone_path = 'yes_drone/*.wav'
no_drone_path = 'unknown/*.wav'

POS = f'{DATASET_PATH}/{yes_drone_path}'
NEG = f'{DATASET_PATH}/{no_drone_path}'
pos = tf.data.Dataset.list_files(POS)
neg = tf.data.Dataset.list_files(NEG)

positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))


In [ ]:
data = positives.concatenate(negatives)
print("No. Wav: ", len(data))
data = data.map(preprocess)
data = data.cache()
data = data.shuffle(3404)
data = data.batch(16)
data = data.prefetch(4)
print("Data: ", len(data))

train = data.take(32)
test = data.skip(32).take(16)

samples, labels = train.as_numpy_iterator().next()  # type: ignore
print(samples.shape)

num_samples = set(int(i.shape[0]) for i in keras.tree.flatten(samples))

No. Wav:  3404
(None, 129)
(None, 129, 1)
Data:  213
(16, 2991, 129, 1)


In [ ]:
model = models.Sequential(
    [
        layers.Input(shape=(1491, 257, 1)),
        layers.Resizing(1024, 256),

        layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu'),
        layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid'),
    ]
)

model.compile('adam', loss='BinaryCrossentropy', run_eagerly=True, metrics=[keras.metrics.Recall(), keras.metrics.Precision()])
model.summary()

hist = model.fit(train, epochs=5, validation_data=test, verbose='auto')


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing (Resizing)             │ (None, 1024, 256, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 1022, 254, 256) │         2,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 1020, 252, 128) │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 510, 126, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 508, 124, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 254, 62, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 252, 60, 32)    │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 483840)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    61,931,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,321,633 (237.74 MB)

 Trainable params: 62,321,633 (237.74 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5


In [ ]:
# Training Stats
plt.title('Loss')
plt.plot(hist.history['loss'], 'r')
plt.plot(hist.history['val_loss'], 'b')
plt.show()

plt.title('Precision')
plt.plot(hist.history['precision'], 'r')
plt.plot(hist.history['val_precision'], 'b')
plt.show()

plt.title('Recall')
plt.plot(hist.history['recall'], 'r')
plt.plot(hist.history['val_recall'], 'b')
plt.show()

print("Test Prediction...")
test = model.predict(samples)
print("Test Prediction result: ", test)